In [93]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import spacy
import urllib.parse as url
import re, string, unicodedata

In [2]:
nlp=spacy.load('en_core_web_sm')

In [3]:
os.chdir('../')

In [67]:
keyword_list = ['asians','blacks']
train_eval = ['evaluation','training']

In [80]:
replace_words = {'Asian American':'Asian_American','high school':'high_school','High school':'High_school',
                'White American':'White_American','Black american':'Black_American',
                'White pe':'White_pe','Black pe':'Black_pe','Black wom':'Black_wom',
                'White wom':'White_wom','Black man':'Black_man','white man':'White_man','CaucAsian':'Caucasian',
                'Black men':'Black_men','White men':'White_men','African American':'African_American',
                'Asian pe':'Asian_pe','native American':'Native_American','Latin American':'Latin_American',
                'violent crime':'violent_crime','Violent crime':'Violent_crime','police brutality':'police_brutality',
                'Police brutality':'Police_brutality','black lives matter':'Black_Lives_Matter',
                'American Indian':'American_Indian','puerto rican':'Puerto_Rican','Puerto rican':'Puerto_Rican',
                'African-Amer':'African_Amer','asian-Amer':'Asian_Amer','Asian-Amer':'Asian_Amer','asian pe':'Asian_pe',
                'asian man':'Asian_man','asian men':'Asian_men','asian wom':'Asian_wom','white wom':'White_wom',
                'Asian man':'Asian_man','Asian men':'Asian_men','Asian wom':'Asian_wom','white men':'White_men',
                'poc ':'POC ','hite supremac':'hite_supremac','brazil':'Brazil','caucAsian':'Caucasian','Asian pe':'Asian_pe',
                'Black lives matter':'Black_Lives_Matter','Native American':'Native_American','CaucAsian':'Caucasian',
                'black men':'Black_men','black wom':'Black_wom','White American':'White_American','black man':'Black_man',
                'Black American':'Black_American',}

In [81]:
ner_words = ['poverty', 'employment', 'homeless', 'job', 'business', 'wealth', 'welfare','educat', 
             'high school', 'college', 'universit', 'school','protest', 'discriminat', 'arrest', 
             'immigra', 'racis', 'segregat', 'communit', 'police brutality', 'blm','stereotyp', 
             'black lives matter', 'violent crime', 'neighborhood', 'police', 'cops','crime', 
             'White man','White men','Whites','Hispanic', 'Latin', 'Latin American', 'Chinese', 
             'Indian', 'Native American', 'Filipino', 'Korean', 'Japanese', 'Vietnamese', 
             'Mexican', 'Puerto Rican', 'Arab', 'Caucasian','Brazilian', ' poc ','Black pe',
             'white supremac', 'African Amer','Asian Amer','White pe','Asian pe','White wom',
             'Black wom','Asian wom','Asian man','Black man','Asian men','Black men','black american',
             'white american', ]
             #'Blacks','Asians',]

In [82]:
ner_words_dict = {'poverty':'ECON', 'employment':'ECON', 'homeless':'ECON', 'job':'ECON','welfare':'ECON',
                  'business':'ECON', 'wealth':'ECON', 'educat':'EDU','high_school':'EDU','school':'EDU',  
                  'college':'EDU', 'universit':'EDU', 'protest':'SCC', 'discriminat':'SCC','stereotyp':'SCC', 
                  'arrest':'SCC','immigra':'SCC', 'racis':'SCC', 'segregat':'SCC', 'communit':'SCC',
                  'police_brutality':'SCC', 'blm':'SCC','black_lives_matter':'SCC','white_supremac':'SCC', 
                  'police':'SCC','cop':'SCC','cops':'SCC','violent_crime':'NBE', 'neighborhood':'NBE','crime':'NBE', 
                  'Asians':'RETH', 'Whites':'RETH', 'Blacks':'RETH', 'Hispanic':'RETH', 'Latin':'RETH',
                  'Latin American':'RETH', 'Chinese':'RETH','Indian':'RETH', 'Native_American':'RETH',
                  'Filipino':'RETH', 'Korean':'RETH', 'Japanese':'RETH', 'Vietnamese':'RETH', 
                  'Mexican':'RETH', 'Puerto_Rican':'RETH', 'Arab':'RETH', 'Caucasian':'RETH', 
                  'n_Amer':'RETH', 'American_Indian':'RETH','Asian_wom':'RETH','Black_American':'RETH',
                  'Black_wom':'RETH','Asian_man':'RETH','Black_man':'RETH','Asian_men':'RETH',
                  'Black_men':'RETH','White_wom':'RETH','Asian_pe':'RETH','Black_pe':'RETH','Brazilian':'RETH',
                  'White_man':'RETH','White_men':'RETH', 'White_pe':'RETH','blm':'SCC','White_American':'RETH',
                  'POC':'RETH', }

In [8]:
def remove_underscore(text):
    """Replace underscore with space if str len is > 4, up to 2 are replaced"""
    if len(str(text)) > 4 and "_" in text:
        return text.replace("_", " ", 2)
    else:
        return text

In [74]:
## Imports sentences from text file and load into a DataFrame
## Changes words with space and replace with underscore - save modified text to seperate text files.
def extract_sents(kw):
    lines = []
    for kl in keyword_list:
        with open(os.getcwd() + r'/data/sentence_samples/%s.txt'%kl) as file_in:
            for line in file_in:
                if kw.casefold() in str(line).casefold():
                    lines.append(line)
    print ('The total number of lines in %s is %s'%(kw,len(lines)))
    df = pd.DataFrame(lines,columns=['sents'])
    df.style.set_properties(subset=["sents"], **{'text-align': 'left'})
    df.sents = df.sents.apply(str).astype('U').values
    for old, new in replace_words.items():
        df.sents = df.sents.str.replace(old, new, regex=False)# continue working on ignorecase
    with open(os.getcwd() + r'/data/sentence_samples/keywords/%s.txt'%kw, 'w') as f:
        np.savetxt(f,df.sents, fmt='%s', newline='')    

In [58]:
## Imports n sentences from text file and load into a DataFrame
##  save 70% sentences as training data, 30% evaluation to respective files/folders.
def extract_sents_partial(kw):
    lines = []
    with open(os.getcwd() + r'/data/sentence_samples/keywords/%s.txt'%kw) as file_in:
        for line in file_in:
            lines.append(line)
    if len(lines) < 1000:
        df = pd.DataFrame(lines,columns=['sents'])
    else:
        df = pd.DataFrame(lines[:1000],columns=['sents'])
    print ('The total number of lines in %s is %s'%(kw,df.shape[0]))
    df.style.set_properties(subset=["sents"], **{'text-align': 'left'})
    df.sents = df.sents.apply(str).astype('U').values

    begin = int((df.shape[0] * 0.7))
    with open(os.getcwd() + r'/data/sentence_samples/keywords/training/%s.txt'%kw, 'w') as f:
        np.savetxt(f,df.sents.iloc[:begin], fmt='%s', newline='') 
    with open(os.getcwd() + r'/data/sentence_samples/keywords/evaluation/%s.txt'%kw, 'w') as f:
        np.savetxt(f,df.sents.iloc[begin:], fmt='%s', newline='')

In [70]:
def combine_files(te):
    lines = []
    for kl in ner_words:
        with open(os.getcwd() + r'/data/sentence_samples/keywords/%s/%s.txt'%(te,kl)) as file_in:
            for line in file_in:
                lines.append(line)
    #print ('The total number of lines in %s is %s'%(kw,len(lines)))
    df = pd.DataFrame(lines,columns=['sents'])
    df.style.set_properties(subset=["sents"], **{'text-align': 'left'})
    df.sents = df.sents.apply(str).astype('U').values
    with open(os.getcwd() + r'/data/sentence_samples/keywords/%s/%s.txt'%(te,te), 'a') as f:
        np.savetxt(f,df.sents, fmt='%s', newline='')

In [59]:
def get_words (keyword): 
    words = []
    with open(os.getcwd() + r'/data/sentence_samples/keywords/%s.txt'%keyword) as file_in:
        for line in file_in:
            doc = nlp(str(line))
            for token in doc:
                if not token.text.isspace():
                    words.append([token.text])
    words = pd.DataFrame(words,columns=['Words'])
    words['Tags'] = 'O'
    for old, new in ner_words_dict.items():
        words.Tags = np.where(words.Words.str.contains(old, case=False), new, words.Tags)
    words.Words = words.Words.apply(remove_underscore).astype('U').values
    words.to_csv(os.getcwd() + r'/data/sentence_samples/keywords/tsv/%s.tsv'%keyword, sep="\t", index=False,  encoding="utf-8")
    # put 'header = False,' if you want to save with no headers

In [95]:
def get_words_combined (keyword): 
    words = []
    with open(os.getcwd() + r'/data/sentence_samples/keywords/%s/%s_1.txt'%(keyword,keyword)) as file_in:
        for line in file_in:
            doc = nlp(str(line))
            for token in doc:
                if not token.text.isspace():
                    words.append([token.text])
    words = pd.DataFrame(words,columns=['Words'])
    words['Tags'] = 'O'
    for old, new in ner_words_dict.items():
        words.Tags = np.where(words.Words.str.contains(old, case=False), new, words.Tags)
    words.Words = words.Words.apply(remove_underscore).astype('U').values
    words.to_csv(os.getcwd() + r'/data/sentence_samples/keywords/%s/tsv/%s_1.tsv'%(keyword,keyword), sep="\t", index=False,  encoding="utf-8")
    # put 'header = False,' if you want to save with no headers

In [83]:
for kw in ner_words:
    extract_sents(kw) #combined

The total number of lines in poverty is 2435
The total number of lines in employment is 1550
The total number of lines in homeless is 271
The total number of lines in job is 3598
The total number of lines in business is 2036
The total number of lines in wealth is 2598
The total number of lines in welfare is 1980
The total number of lines in educat is 4439
The total number of lines in high school is 738
The total number of lines in college is 3322
The total number of lines in universit is 1533
The total number of lines in school is 6143
The total number of lines in protest is 1359
The total number of lines in discriminat is 7659
The total number of lines in arrest is 1948
The total number of lines in immigra is 5373
The total number of lines in racis is 38798
The total number of lines in segregat is 2322
The total number of lines in communit is 4437
The total number of lines in police brutality is 570
The total number of lines in blm is 683
The total number of lines in stereotyp is 5133

In [84]:
for kw in ner_words:
    extract_sents_partial(kw) #combined

The total number of lines in poverty is 1000
The total number of lines in employment is 1000
The total number of lines in homeless is 271
The total number of lines in job is 1000
The total number of lines in business is 1000
The total number of lines in wealth is 1000
The total number of lines in welfare is 1000
The total number of lines in educat is 1000
The total number of lines in high school is 738
The total number of lines in college is 1000
The total number of lines in universit is 1000
The total number of lines in school is 1000
The total number of lines in protest is 1000
The total number of lines in discriminat is 1000
The total number of lines in arrest is 1000
The total number of lines in immigra is 1000
The total number of lines in racis is 1000
The total number of lines in segregat is 1000
The total number of lines in communit is 1000
The total number of lines in police brutality is 570
The total number of lines in blm is 683
The total number of lines in stereotyp is 1000


In [89]:
for i in train_eval:
    os.remove(os.getcwd() + r'/data/sentence_samples/keywords/%s/%s.txt'%(i,i))
    combine_files(i)

In [54]:
for kw in ner_words[41:42]:
    extract_sents(kw) #Blacks - for testing

The total number of lines in Puerto Rican is 191


In [ ]:
for kw in ner_words:
    extract_sents(kw) #asians

In [402]:
for k in ner_words[:1]:
    get_words(k)

In [96]:
for k in train_eval:
    get_words_combined(k)

In [97]:
for k in train_eval:
    df = pd.read_csv(os.getcwd() + r'/data/sentence_samples/keywords/%s/tsv/%s_1.tsv'%(k,k), sep="\t", encoding="utf-8")
    df1= df.groupby(['Tags'])['Tags'].count().to_frame(name = '# of Ent Types').reset_index()
    print(df1)
    #total = df1['Entity Label'].count()

   Tags  # of Ent Types
0  ECON            2173
1   EDU            1753
2   NBE            1324
3     O          318657
4  RETH           25657
5   SCC            6085
   Tags  # of Ent Types
0  ECON            5072
1   EDU            4353
2   NBE            2791
3     O          753571
4  RETH           63314
5   SCC           14003
